In [1]:
from Tweets import Tweets
import pandas as pd
from tqdm.notebook import tqdm
from decouple import config

tweets = Tweets(config('TWITTER_API_KEY'),
               config('TWITTER_API_SECRET'),
               config('TWITTER_ACCESS_TOKEN_KEY'),
               config('TWITTER_ACCESS_TOKEN_SECRET'))

languages = {
                1: 'en',
                2: 'es',
                3: 'fr',
                4: 'de',
                5: 'nl',
                6: 'it',
            }

months = ['December', 'January', 'February', 'March', 'April', 'May']

In [2]:
for month in tqdm(months):
    for day in tqdm([0, 1, 2, 3, 4]):
        df = pd.read_json('Data/FilteredTwitterIDs/' + str(month) + str(day) + '.json')
        ids = df.tweet_id.values.tolist()
        for j in languages:
            for k in range(1,11):
                tweets.addTweets(ids[((j-1)*1000)+(k-1)*100:((j-1)*1000)+k*100])
            tweets.preProcess(languages[j])
            if j != 1:
                tweets.translate(language=languages[j])
            tweets.saveJSON('Data/Text/' + str(month) + str(day) + languages[j])
            tweets.reset()



  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

KeyError: 'nl'